In [1]:
import os
import urllib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf

from bs4 import BeautifulSoup

### Get data

In [2]:
coordinates = pd.read_csv('data/photo2gps.txt', header=None, delimiter=' ', names=['longitude', 'latitude'], index_col=0)
coordinates.index = coordinates.index.map(lambda s: tuple(s.split('/')))
coordinates.head(2)

longitude    latitude
airplane 1001224523.jpg   31.349944 -105.928990
         10030517043.jpg  43.123688  -77.623395

In [3]:
urls = pd.read_csv('data/gps_urls.txt', header=None, delimiter=' ', names=['link'])
urls.head(2)

,link
43398,https://www.flickr.com/photos/george/43398/
58665,https://www.flickr.com/photos/swanksalot/58665/


In [4]:
def get_tags(url):
    tags = []
    try:
        u2 = urllib.request.urlopen(url)  
        soup = BeautifulSoup(u2)

#     for image_src in soup.find_all("title"):
#         title = image_src

        for image_src in soup.find_all("meta"):
            if image_src.get('name') == 'keywords':
                tags = image_src.get('content').split(', ')
    except:
        tags = []
    return tags

In [5]:
import tqdm.notebook as tqdm
import dask.dataframe as dd

In [6]:
import multiprocessing

multiprocessing.cpu_count()

32

In [12]:
def split_dataframe(df, starting_chunk=0, chunk_size = 10000): 
    chunk = None
    num_chunks = len(df) // chunk_size + 1
    for i in np.arange(starting_chunk, num_chunks):
        chunk = df.iloc[i*chunk_size:(i+1)*chunk_size]
        yield chunk

In [13]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [15]:
for i, chunk in tqdm.tqdm(enumerate(split_dataframe(urls, 0, 1000))):
    tagged_chunk = chunk.link.map(get_tags)
    tagged_chunk.to_csv(f'tags/chunk_{i}')

0it [00:00, ?it/s]